In [3]:
import numpy as np
import pandas as pd 
from sklearn.metrics import auc

In [4]:
datafile = pd.read_csv("C:\\Users\\namil\\Downloads\\CaM01-001_CGM_Libre.csv")
time = datafile['mins'].to_numpy()
bg = datafile['BG'].to_numpy()

In [5]:
time = time[~np.isnan(time)]
bg = bg[~np.isnan(bg)]
area = auc(time, bg)
print(area)

145717.5


In [6]:
tuples = zip(time,bg)
arr = np.asarray(list(tuples))

#pick a time interval
filtered = arr[arr[:,0] <= 660]

#calculate iauc for that time interval
area = auc(filtered[:,0], filtered[:,1])
print(area)

2512.5
